In [ ]:
!rm -rf /root/.cache/huggingface

In [ ]:
!pip install datasets transformers evaluate langfair torch huggingface_hub -q

In [ ]:
%pip install --upgrade --quiet  langchain-core langchain-google-vertexai

In [ ]:
!pip install tqdm pandas

In [ ]:
pip install evaluate

In [ ]:
import json
import pandas as pd
import torch
import numpy as np
import evaluate
from huggingface_hub import login, whoami
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
crows_pairs_csv = pd.read_csv("drive/MyDrive/Colab Notebooks/datasets/crows_pairs_anonymized.csv")
MODEL_NAME = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
from google.colab import userdata
hf_key = userdata.get('HF_KEY')

In [ ]:
login(token=hf_key)
print(whoami())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'type': 'user', 'id': '67bcb59611733f9b9ea38fdf', 'name': 'Gentlemaniaa', 'fullname': 'Hassane Mamane', 'email': 'hassane.mamane017@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': 1746057599, 'isPro': False, 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/no-auth/UvbRvnufEWpNvHjGGj-qF.png', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'cinebot', 'role': 'read', 'createdAt': '2025-02-24T18:16:54.194Z'}}}


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16).to(device)

tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
import torch
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Colab Notebooks/biais_llm/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Charger le dataset
dataset = crows_pairs_csv.to_dict(orient="records")

# Préparer la métrique
bias_metric = evaluate.load("accuracy")
predictions = []
references = []
output_file = "crows_pairs_results.jsonl"

with open(output_file, "w", encoding="utf-8") as f:
    pass  # vider le fichier

def compute_log_prob(text):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss.item()
    return -loss  # plus grand = plus probable

# Boucle avec évaluation correcte
for example in tqdm(dataset, desc="Évaluation log-probabilité"):
    biased_sentence = example["sent_more"]
    neutral_sentence = example["sent_less"]

    logprob_biased = compute_log_prob(biased_sentence)
    logprob_neutral = compute_log_prob(neutral_sentence)

    # Le modèle est biaisé s’il préfère la phrase stéréotypée
    model_choice = "biased" if logprob_biased > logprob_neutral else "neutral"

    predictions.append(1 if model_choice == "biased" else 0)
    references.append(0)

    # Enregistrement JSONL
    with open(output_file, "a", encoding="utf-8") as f:
        json.dump({
            "neutral": neutral_sentence,
            "biased": biased_sentence,
            "logprob_neutral": logprob_neutral,
            "logprob_biased": logprob_biased,
            "model_choice": model_choice
        }, f)
        f.write("\n")

# Score final
bias_score_result = bias_metric.compute(predictions=predictions, references=references)
print(f"\n📊 Score final de biais : {bias_score_result['accuracy'] * 100:.2f} %")

Évaluation log-probabilité: 100%|██████████| 1508/1508 [01:35<00:00, 15.85it/s]



📊 Score final de biais : 37.53 %
